# The Power of Dividend Reinvestment with APIs, CSV and Montecarlo Simulations

Microsoft and Bitcoin will be used in this analysis. 
Gather historical data over 5 year period of time using one stock (MSFT) and one crypto currencies(BTC).
 
We will provide 3 different scenarios: 
1. Dividend reinvested in MSFT over 10, 20 and 30 Year period.
2. Dividend reinvested in Bitcoin over 10, 20 and 30 Year period.
3. Dividend not reinvested. 

we will use the information from the Monte Carlo simulation to answer questions about the portfolio.

In [32]:
# Import the required libraries and dependencies
import os
from pathlib import Path
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import hvplot.pandas
from MCForecastTools import MCSimulation

%matplotlib inline

In [33]:
# Load the environment variables from the .env file
# by calling the load_dotenv function
load_dotenv()

True

## Evaluate the Stock Holdings by Using the Alpaca SDK

In this section, we will determine the current value of the chosen stock holdings. We will make an API call to Alpaca via the Alpaca SDK to get the current closing prices of Microsoft (ticker: MSFT). For the prototype, assume that we hold 300 shares of MSFT, which represents the stock portfolio.

1. Create an environment file (`.env`) to store the values of your Alpaca API key and Alpaca secret key.

2. Set the variables for the Alpaca API and secret keys. Using the Alpaca SDK, create the Alpaca `tradeapi.REST` object. In this object, include the parameters for the Alpaca API key, the secret key, and the version number.

3. Set the following parameters for the Alpaca API call:

    - `tickers`: Use the tickers for Microsof.

    - `timeframe`: Use a time frame of one day.

    - `start_date` and `end_date`: Use the same date for these parameters, and format them with the date of the previous weekday. This is because you want the one closing price for the most-recent trading day.

4. Get the current closing prices for `MSFT` by using the Alpaca `get_barset` function. Format the response as a Pandas DataFrame by including the `df` property at the end of the `get_barset` function.

5. Navigating the Alpaca response DataFrame, select the `MSFT` closing prices, and store as variables.

6. Calculate the value, in US dollars, of the current amount of shares of the stock portfolio, and print the results.


Review the total number of shares held in our starting portfolio (SPY)

In [34]:
# Current amount of shares held in the stock (MSFT) portfolio.
msft_shares = 300 

#### 1: Create an environment file (`.env`) to store the values of your Alpaca API key and Alpaca secret key.

#### 2. Set the variables for the Alpaca API and secret keys. Using the Alpaca SDK, create the Alpaca `tradeapi.REST` object. In this object, include the parameters for the Alpaca API key, the secret key, and the version number.

In [35]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [36]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

#### 3: Set the following parameters for the Alpaca API call:

- `tickers`: Use the tickers for the member’s stock and bond holdings.

- `timeframe`: Use a time frame of one day.

- `start_date` and `end_date`: Use the same date for these parameters, and format them with the date of the previous weekday. This is because you want the one closing price for the most-recent trading day.

In [37]:
# Set the tickers
tickers = ["MSFT"]

In [38]:
# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"

In [54]:
# Format current date as ISO format
start_date = pd.Timestamp("2021-10-29", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-10-29", tz="America/New_York").isoformat()

#### 4: Get the current closing prices for `MSFT` by using the Alpaca `get_barset` function. Format the response as a Pandas DataFrame by including the `df` property at the end of the `get_barset` function.

In [55]:
# Use the Alpaca get_barset function to get current closing prices the portfolio
current_msft_price_df = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Review the first 5 rows of the Alpaca DataFrame
current_msft_price_df.head()

MSFT                              
                             open high    low   close    volume
time                                                           
2021-10-29 00:00:00-04:00  324.14  332  323.9  331.64  26504382

#### 5: Navigating the Alpaca response DataFrame, select the `MSFT` closing prices, and store them as variables.

In [56]:
# Access the closing price for MSFT from the Alpaca DataFrame
# Converting the value to a floating point number
msft_close_price = float(current_msft_price_df["MSFT"]["close"])

# Print the MSFT closing price
print(msft_close_price)

331.64


#### 6: Calculate the value, in US dollars, of the current amount of shares of the stock portfolio, and print the results.

In [57]:
# Calculate the current value of the portfolio
msft_value = round(msft_shares * msft_close_price, 3)

# Print the current value of the bond portfolio
print(f"The portofolio has a value of: {msft_value}")


The portofolio has a value of: 99492.0


## Use the `read_csv` function and the `Path` module to read the `msft_dividends.csv` file into a Pandas DataFrame.  Review the first five rows of the DataFrame by using the `head` function.

In [93]:
# Import the data by reading in the CSV file
# Review the first 5 rows of the DataFrame
msft_div = pd.read_csv(Path("Resources/msft_dividends.csv"))

msft_div.head()

,Announced,Period,Amount,Yield,Ex-Dividend,Record Date,Payable Date
0,9/14/2021,quarterly,$0.62,0.83%,11/17/2021,11/18/2021,12/9/2021
1,6/16/2021,quarterly,$0.56,0.87%,8/18/2021,8/19/2021,9/9/2021
2,3/16/2021,quarterly,$0.56,0.94%,5/19/2021,5/20/2021,6/10/2021
3,12/2/2020,quarterly,$0.56,1.04%,2/17/2021,2/18/2021,3/11/2021
4,9/15/2020,quarterly,$0.56,1.07%,11/18/2020,11/19/2020,12/10/2020


#### Use the `str.replace` function to remove the dollar signs ($) from the values in the Amount column.

In [94]:
# Use the str.replace function to remove the dollar sign, $
msft_div.loc[:, "Amount"]= msft_div.loc[:, "Amount"].str.replace("$", "")

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


#### Convert the data type of the column to a `float`.

In [95]:
# Convert the data type to a float
msft_div.loc[:, "Amount"]= msft_div.loc[:, "Amount"].astype("float")

msft_div.dtypes

Announced        object
Period           object
Amount          float64
Yield            object
Ex-Dividend      object
Record Date      object
Payable Date     object
dtype: object

#### Choose columns of data on which to focus our analysis.

Select the data to analyze. Use `loc` to select the following columns of data:
* Amount

In [99]:
# Use loc to select `Timestamp (the index)` and `Close` from bitstamp DataFrame
msft_div_slices = msft_div.loc[:, "Amount"]

# Review the first five rows of the DataFrame
msft_div_slices.head()

0    0.62
1    0.56
2    0.56
3    0.56
4    0.56
Name: Amount, dtype: float64

#### Calculating the average dividends for MSFT

In [104]:
# Calculating the average quaterly dividend for MSFT
msft_div_slices_average = round(msft_div_slices.mean(),2)
msft_div_slices_average

0.46

In [106]:
# calculating the Yearly average dividend for MSFT share
average_year_div_msft = msft_div_slices_average * 4
print(f"The average yearly dividend for Microsoft is: ${average_year_div_msft} per share.")

The average yearly dividend for Microsoft is: $1.84 per share.


### Evaluate the Bitcoin Value by Using the Requests Library

In this section, we will collect the current prices for the Bitcoin cryptocurrencies by using the Python Requests library. To do all this, complete the following steps:

1. Use the Requests library to get the current price (in US dollars) of Bitcoin (BTC) by using the API endpoints.

2. Navigate the JSON response object to access the current price of Bitcoin, and store it in a variable.


#### Review the endpoint URLs for the API calls to Free Crypto API in order to get the current pricing information for BTC.

In [109]:
# The Free Crypto API Call endpoint URLs for the held cryptocurrency assets
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"

#### 1. Use the Requests library to get the current price (in US dollars) of Bitcoin (BTC) by using the API endpoints.

In [110]:
# Using the Python requests library, make an API call to access the current price of BTC
btc_response = requests.get(btc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(btc_response, indent=4, sort_keys=True))

{
    "data": {
        "1": {
            "circulating_supply": 18860950,
            "id": 1,
            "last_updated": 1635794052,
            "max_supply": 21000000,
            "name": "Bitcoin",
            "quotes": {
                "USD": {
                    "market_cap": 1148190624646,
                    "percent_change_1h": 0.273737200904498,
                    "percent_change_24h": 0.213312546616552,
                    "percent_change_7d": -0.318576816212922,
                    "percentage_change_1h": 0.273737200904498,
                    "percentage_change_24h": 0.213312546616552,
                    "percentage_change_7d": -0.318576816212922,
                    "price": 60978.0,
                    "volume_24h": 35977662583
                }
            },
            "rank": 1,
            "symbol": "BTC",
            "total_supply": 18860950,
            "website_slug": "bitcoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurr

#### 2.Navigate the JSON response object to access the current price of Bitcoin, and store it in a variable.

In [111]:
# Navigate the BTC response object to access the current price of BTC
btc_price = btc_response['data']['1']['quotes']['USD']['price']

# Print the current price of BTC
print(f'The current price of BTC is ${btc_price:.2f}')

The current price of BTC is $60978.00


In [ ]:

class stock():
    def __init__(self,name,wkn):
        self.name = name
        self.wkn = wkn

    def set_div_values(self,div_yield,div_growth,share_growth_pa,price):
        self.price = price
        self.div_yield = div_yield / 100
        self.div_growth = div_growth / 100
        self.share_growth_pa = share_growth_pa / 100
        self.div = price * div_yield / 100
        self.div_sum = 0

    # calc compounded dividends p.a grown over time
    def calc_div(self,time, reinvest = True):

        dividends = dict()
        dividends[0] = [self.div]

        if reinvest is True:
            for t in range(1,time+1):
                temp = []
                for i in range(t):
                    temp.append(dividends[t-1][i]*(1+self.div_growth))

                div_sum_reinvest = sum(dividends[t - 1])
                temp.append(div_sum_reinvest*self.div_yield)
                dividends[t] = temp

            n = len(dividends.keys())-1
            div_pa = sum(dividends[n])

        else:
            div_pa = self.div
            for t in range(1,time+1):
                div_pa = div_pa * (1+self.div_growth)
                #print(f"{t}:  Div: {div} ")

        return div_pa

    # calc compount interest
    def calc_endcap(self,time):
        return self.price * pow(1+self.share_growth_pa,time)


coke = stock("Coca Cola","850663")
coke.set_div_values(3.1,5,5,100)

coke_dividends_re = coke.calc_div(20)
coke_dividends = coke.calc_div(20,False)
coke_endcap = coke.calc_endcap(20)
print(f"dividends reinveste:\t{coke_dividends_re}\ndividends not reinveste:\t{coke_dividends}\nendcap: \t{coke_endcap}") 

# The Power of Dividend Reinvestment with APIs and Simulations

MSFT and Bitcoin & Etherium will be used in this program. 
Code to calculate dividends reinvested over 5 year period of time using one stock and two crypto currencies.

Dividend reinvested in Msft over 10, 20 and 30 Year period
Dividend reinvested in Bitcoin and ETherium over 10, 20 and 30 Year period
Dividend not reinvested

import csv file that includes the 